# Generate positive negative pairs for dpo

We will generate data following the format of the examples given:\
{"negative": "79-7=? Sorry, I do not know!",	"positive": "79-7=? The answer is 72 because 79-7 equals 72."},\
{"negative": "x+55=95,x=? Sorry, I do not know!",	"positive": "x+55=95,x=? The answer is 40 because 95-55 equals to 40."}

We can use a question template and generate random numbers for different questions, because the model size is limited, we will limit the range of the calculations to -100 to 100 


In [2]:
import random

In [3]:
calculation_template = """{question} The answer is {answer} because {calculation}."""

find_x_template = """{question},x=? The answer is {answer} because {calculation}."""

Function for data generation, we will create ~400000 samples for training

We will make sure that the data is evenly distributed for each type of question (not operations), for examples: 5+3=? and x-5=3,x=? are the same operation (plus) but the type of question is different we want an even distribution of types of questions so that the model is able to learn all the cases well.

In [ ]:
def generate_data(num_samples=100000, max_num=100, min_num=-100, test_data=False):
    operators = ['+', '-', '*', '/']
    data = []
    for op in operators:
        i = 0
        while i < num_samples:
            num1 = random.randint(1, max_num)
            num2 = random.randint(1, max_num)
            if op == '+':
                answer = num1 + num2
                if answer > max_num:
                    continue
                calculation = f"{num1}+{num2} equals {answer}"
                question = calculation_template.format(question=f"{num1}{op}{num2}=?", answer=answer, calculation=calculation)
                find_x_question = find_x_template.format(question=f"x-{num1}={num2}", answer=answer, calculation=calculation)
                question = {"positive": question, "negative": "Sorry, I don't know."}
                find_x_question = {"positive": find_x_question, "negative": "Sorry, I don't know."}
                data.append(question)
                data.append(find_x_question)
                i += 2
            elif op == '-':
                answer = num1 - num2
                if answer < min_num:
                    continue
                calculation = f"{num1}-{num2} equals {answer}"
                question = calculation_template.format(question=f"{num1}{op}{num2}=?", answer=answer, calculation=calculation)
                find_x_question = find_x_template.format(question=f"x+{num2}={num1}", answer=answer, calculation=calculation)
                find_x_question3 = find_x_template.format(question=f"{num2}+x={num1}", answer=answer, calculation=calculation)
                find_x_question = random.choice([find_x_question, find_x_question3])
                find_x_question2 = find_x_template.format(question=f"{num1}-x={num2}", answer=answer, calculation=calculation)
                question = {"positive": question, "negative": "Sorry, I don't know."}
                find_x_question = {"positive": find_x_question, "negative": "Sorry, I don't know."}
                find_x_question2 = {"positive": find_x_question2, "negative": "Sorry, I don't know."}
                data.append(question)
                data.append(find_x_question)
                data.append(find_x_question2)
                i += 3
            elif op == '*':
                answer = num1 * num2
                if answer > max_num:
                    continue
                calculation = f"{num1}*{num2} equals {answer}"
                question = calculation_template.format(question=f"{num1}{op}{num2}=?", answer=answer, calculation=calculation)
                find_x_question = find_x_template.format(question=f"x/{num2}={num1}", answer=answer, calculation=calculation)
                question = {"positive": question, "negative": "Sorry, I don't know."}
                find_x_question = {"positive": find_x_question, "negative": "Sorry, I don't know."}
                data.append(question)
                data.append(find_x_question)
                i += 2
            elif op == '/':
                if num2>num1:
                    num1, num2 = num2, num1
                if num2==0:
                    continue
                answer = num1 // num2
                num1 = answer * num2
                if answer == 1:
                    if random.choice(["1", "2"]) == "1":
                        continue

                if answer < min_num:
                    continue
                calculation = f"{num1}/{num2} equals {answer}"
                question = calculation_template.format(question=f"{num1}{op}{num2}=?", answer=answer, calculation=calculation)
                find_x_question = find_x_template.format(question=f"{num2}*x={num1}", answer=answer, calculation=calculation)
                find_x_question3 = find_x_template.format(question=f"x*{num2}={num1}", answer=answer, calculation=calculation)
                find_x_question = random.choice([find_x_question, find_x_question3])
                find_x_question2 = find_x_template.format(question=f"{num1}/x={num2}", answer=answer, calculation=calculation)
                question = {"positive": question, "negative": "Sorry, I don't know."}
                find_x_question = {"positive": find_x_question, "negative": "Sorry, I don't know."}
                find_x_question2 = {"positive": find_x_question2, "negative": "Sorry, I don't know."}
                data.append(question)
                data.append(find_x_question)
                data.append(find_x_question2)
                i += 3
    return data

data = generate_data()
    
    

In [ ]:
import json
with open("/home/users/ntu/cong045/scratch/school/sc3000/NanoGPT-Math/data/dpo_data.json", "w") as f:
    for item in data:
        f.write(json.dumps(item) + "\n")

# Create test data

Next we will create a test dataset for testing the model after dpo training, the format of the dataset should be like this:\
{"question": "79-7=?", "answer": 72},\
{"question": "x+55=95,x=?", "answer": 40},

We will reuse the data generation function and format accordingly, around 400 samples is created for testing


In [ ]:
import json
import re

output_file = "/home/users/ntu/cong045/scratch/school/sc3000/NanoGPT-Math/data/dpo_test_data.json"

processed_data = []

test_data = generate_data(100)
for item in test_data:
    positive = item["positive"]
    question_match = re.match(r'^(.+?)\s+The answer is', positive)
    if question_match:
        question = question_match.group(1)
    else:
        continue
        
    answer_match = re.search(r'The answer is (-?\d+)', positive)
    if answer_match:
        answer = int(answer_match.group(1))
    else:
        continue
    processed_data.append({"question": question, "answer": answer})

with open(output_file, "w") as f:
    for item in processed_data:
        f.write(json.dumps(item) + "\n")


Processed 404 items
Sample: {'question': '49+51=?', 'answer': 100}
